In [ ]:
# Cell 1 – imports, mount Drive, paths

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')

# CHANGE THIS to where your CSVs live
DATA_ROOT = "/content/drive/MyDrive/deep_triage"

CLASSIFIER_FILE = f"{DATA_ROOT}/classifier_data_20.csv"   # we use the 20-threshold set
DEEP_FILE       = f"{DATA_ROOT}/deep_data.csv"            # not strictly needed, but OK to keep
print("Classifier file:", CLASSIFIER_FILE)
print("Deep file      :", DEEP_FILE)


KeyboardInterrupt: 

In [ ]:
# Cell 2 – load classifier_data_20 and build a single text field

clf_df = pd.read_csv(CLASSIFIER_FILE)

print("Columns:", list(clf_df.columns))
print("Number of labelled bugs:", len(clf_df))

# We assume the columns are: owner, issue_title, description
for c in ["owner", "issue_title", "description"]:
    if c not in clf_df.columns:
        raise ValueError(f"Expected column '{c}' not found in classifier_data_20.csv")

# Text = title + description
clf_df["text"] = (
    clf_df["issue_title"].fillna("").astype(str)
    + " "
    + clf_df["description"].fillna("").astype(str)
)

# Encode developers as integer IDs
clf_df["dev_id"] = clf_df["owner"].astype("category").cat.codes
dev_ids = clf_df["dev_id"].values
n_devs = clf_df["dev_id"].nunique()

print("Number of developers:", n_devs)
clf_df[["owner", "dev_id"]].head()


In [ ]:
# Cell 3 – tokenisation + splits

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_VOCAB = 50000
MAX_LEN   = 300      # you can tune; DBRNN used long sequences
BATCH_SIZE = 256
VAL_SIZE = 0.1       # 10% of data (out of 80% train+val)

texts = clf_df["text"].tolist()
labels = dev_ids

# Tokeniser on all text
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
seqs = tokenizer.texts_to_sequences(texts)
X_all = pad_sequences(seqs, maxlen=MAX_LEN, padding="post", truncating="post")
y_all = np.array(labels, dtype="int32")

vocab_size = min(MAX_VOCAB, len(tokenizer.word_index) + 1)
print("Vocab size (capped):", vocab_size)
print("X_all shape:", X_all.shape)

# Stratified 80/20 train+test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_all, y_all,
    test_size=0.2,
    random_state=42,
    stratify=y_all,
)

# From train_full, carve out validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full,
    test_size=VAL_SIZE,
    random_state=42,
    stratify=y_train_full,
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


In [ ]:
# Cell 4 – DBRNN model definition (BiLSTM + simple attention) – FIXED

EMBED_DIM   = 128
LSTM_UNITS  = 128

def build_dbrnn_model(vocab_size, max_len, n_classes):
    inputs = layers.Input(shape=(max_len,), dtype="int32")

    # IMPORTANT: mask_zero=False to avoid mask bugs with attention
    x = layers.Embedding(
        input_dim=vocab_size,
        output_dim=EMBED_DIM,
        mask_zero=False
    )(inputs)

    # Bi-LSTM over the sequence
    x = layers.Bidirectional(
        layers.LSTM(LSTM_UNITS, return_sequences=True)
    )(x)   # shape: (B, T, 2*LSTM_UNITS) = (B, T, 256)

    # --- Simple additive attention ---

    # Unnormalised attention scores e_t
    e = layers.Dense(1, activation="tanh")(x)           # (B, T, 1)

    # Normalise over the time dimension T
    alpha = layers.Lambda(
        lambda t: tf.nn.softmax(t, axis=1),
        name="attn_weights"
    )(e)                                                # (B, T, 1)

    # Apply attention and sum over time: sum_t alpha_t * h_t
    def apply_attention(tensors):
        h, a = tensors   # h: (B, T, 256), a: (B, T, 1)
        return tf.reduce_sum(h * a, axis=1)   # (B, 256)

    doc_repr = layers.Lambda(apply_attention, name="doc_repr")([x, alpha])

    # Final classifier over developers
    outputs = layers.Dense(n_classes, activation="softmax")(doc_repr)

    model = models.Model(inputs=inputs, outputs=outputs, name="DBRNN")
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

n_classes = n_devs
model = build_dbrnn_model(vocab_size, MAX_LEN, n_classes)
model.summary()


In [ ]:
# Cell 5 – train DBRNN

EPOCHS = 10

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    callbacks=callbacks,
)


In [ ]:
# Cell 6 – DBRNN baseline evaluation (Top-N Hit Ratio)

def hit_at_k(probs, y_true, k):
    topk = np.argsort(-probs, axis=1)[:, :k]
    hits = (topk == y_true.reshape(-1, 1)).any(axis=1)
    return float(hits.mean())

# Probabilities on test set
probs_base = model.predict(X_test, batch_size=BATCH_SIZE)


print("=== DBRNN baseline (no graph) ===")

def hit_at_k(probs, y_true, k):
    topk = np.argsort(-probs, axis=1)[:, :k]
    hits = (topk == y_true.reshape(-1, 1)).any(axis=1)
    return hits.mean()

probs = model.predict(X_test, batch_size=BATCH_SIZE)

TOPKS = [1, 3, 5, 7, 9]
hrs_dbrnn = []

for k in TOPKS:
    hr = hit_at_k(probs, y_test, k)
    hrs_dbrnn.append(hr)
    print(f"Top-{k} Hit Ratio: {hr:.4f}")

avg_hr = np.mean(hrs_dbrnn)
print(f"Average Hit Ratio DBRNN : {avg_hr:.4f}")




In [ ]:
# Cell 7 – developer representations + adjacency matrix

# 7.1 – bug representations from the "doc_repr" layer
repr_model = models.Model(
    inputs=model.input,
    outputs=model.get_layer("doc_repr").output
)

bug_repr_all = repr_model.predict(X_all, batch_size=BATCH_SIZE)
repr_dim = bug_repr_all.shape[1]
print("Bug repr shape:", bug_repr_all.shape)

# 7.2 – aggregate to developer-level vectors (mean of their bugs)
dev_repr = np.zeros((n_devs, repr_dim), dtype=np.float32)
dev_counts = np.zeros(n_devs, dtype=np.int32)

for i, d in enumerate(y_all):
    dev_repr[d] += bug_repr_all[i]
    dev_counts[d] += 1

nonzero = dev_counts > 0
dev_repr[nonzero] /= dev_counts[nonzero][:, None]

print("Developer repr shape:", dev_repr.shape)

# 7.3 – cosine similarity graph + k-NN sparsification
K_NEIGHBORS = 10

sim = cosine_similarity(dev_repr)
np.fill_diagonal(sim, 0.0)

A = np.zeros_like(sim, dtype=np.float32)
for i in range(n_devs):
    idx = np.argsort(-sim[i])[:K_NEIGHBORS]
    A[i, idx] = sim[i, idx]

# Symmetrise
A = np.maximum(A, A.T)

# 7.4 – row-normalise (random-walk matrix)
row_sums = A.sum(axis=1, keepdims=True) + 1e-8
A_norm = A / row_sums

print("A_norm shape:", A_norm.shape)


In [ ]:
# Cell 8 – GRCNN-style dev-graph smoothing on top of DBRNN probabilities



print("=== GRCNN-style (DBRNN + developer graph smoothing) ===")

probs_graph = probs_base.copy()
for _ in range(STEPS):
    probs_graph = np.matmul(probs_graph, A_norm)

TOPKS = [1, 3, 5, 7, 9]
hrs_grcnn = []

for k in TOPKS:
    hr_g = hit_at_k(probs_graph, y_test, k)
    hrs_grcnn.append(hr_g)
    print(f"Top-{k} Hit Ratio: {hr_g:.4f}")

avg_hr_graph = np.mean(hrs_grcnn)
print(f"Average Hit Ratio GRCNN : {avg_hr_graph:.4f}")

